In [ ]:
import os
import re
import sys
import dagger
from dagger import dag
from dotenv import load_dotenv
from anthropic import Anthropic
from workspace import Workspace
from models.anthropic import AsyncAnthropic
from models.utils import loop_completion

In [2]:
PROMPT = """
Generate a PostgREST application using HTMX entirely in SQL.

Rules:
- Schema should be named "api".
- Always use the "text/html" domain for HTML content.
- Grant necessary permissions to the "web_anon" role.


Example simple TODO App:
<sql>
create schema api;

create table api.todos (
  id int primary key generated by default as identity,
  done boolean not null default false,
  task text not null,
  due timestamptz
);

insert into api.todos (task) values ('finish tutorial 0'), ('pat self on back');

create role web_anon nologin;
grant usage on schema api to web_anon;
grant select on api.todos to web_anon;
</sql>


Example simple HTMX app:
<sql>
create schema api;

create table api.todos (
  id int primary key generated by default as identity,
  done boolean not null default false,
  task text not null,
  due timestamptz
);

create domain "text/html" as text;

create or replace function api.sanitize_html(text) returns text as $$
  select replace(replace(replace(replace(replace($1, '&', '&amp;'), '"', '&quot;'),'>', '&gt;'),'<', '&lt;'), '''', '&apos;')
$$ language sql;

create or replace function api.html_todo(api.todos) returns text as $$
  select format($html$
    <div>
      <%2$s>
        %3$s
      </%2$s>
    </div>
    $html$,
    $1.id,
    case when $1.done then 's' else 'span' end,
    api.sanitize_html($1.task)
  );
$$ language sql stable;

create or replace function api.html_all_todos() returns text as $$
  select coalesce(
    string_agg(api.html_todo(t), '<hr/>' order by t.id),
    '<p><em>There is nothing else to do.</em></p>'
  )
  from api.todos t;
$$ language sql;

create or replace function api.add_todo(_task text) returns "text/html" as $$
  insert into api.todos(task) values (_task);
  select api.html_all_todos();
$$ language sql;

create or replace function api.index() returns "text/html" as $$
  select $html$
    <!DOCTYPE html>
    <html>
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
      <title>PostgREST + HTMX To-Do List</title>
      <!-- Pico CSS for CSS styling -->
      <link href="https://cdn.jsdelivr.net/npm/@picocss/pico@next/css/pico.min.css" rel="stylesheet"/>
      <!-- htmx for AJAX requests -->
      <script src="https://unpkg.com/htmx.org"></script>
    </head>
    <body>
      <main class="container"
            style="max-width: 600px"
            hx-headers='{"Accept": "text/html"}'>
        <article>
          <h5 style="text-align: center;">
            PostgREST + HTMX To-Do List
          </h5>
          <form hx-post="/rpc/add_todo"
                hx-target="#todo-list-area"
                hx-trigger="submit"
                hx-on="htmx:afterRequest: this.reset()">
            <input type="text" name="_task" placeholder="Add a todo...">
          </form>
          <div id="todo-list-area">
            $html$
              || api.html_all_todos() ||
            $html$
          <div>
        </article>
      </main>
    </body>
    </html>
  $html$;
$$ language sql;

create role web_anon nologin;
grant usage on schema api to web_anon;
grant all on api.todos to web_anon;
</sql>


Return application code within <sql> tags.
Generate an application using PostgREST and HTMX for the following request:
""".strip()

In [3]:
load_dotenv()
client = Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])

In [4]:
req_prompt = PROMPT + "\nApp To track my coffee consumption.\n"

In [7]:
response = message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=4096,
    messages=[{"role": "user", "content": req_prompt}],
)

In [10]:
print(response.content[0].text)

I'll create a PostgREST application with HTMX to track coffee consumption. The app will allow users to log their coffee drinks with details like type, size, caffeine content, and timestamps.

<sql>
-- Create our API schema
create schema api;

-- Create tables for coffee tracking
create table api.coffee_types (
  id int primary key generated by default as identity,
  name text not null unique,
  caffeine_mg_per_oz int not null
);

create table api.cup_sizes (
  id int primary key generated by default as identity,
  name text not null unique,
  size_oz int not null
);

create table api.coffee_log (
  id int primary key generated by default as identity,
  coffee_type_id int not null references api.coffee_types(id),
  cup_size_id int not null references api.cup_sizes(id),
  consumed_at timestamptz not null default now(),
  notes text
);

-- Insert some initial data
insert into api.coffee_types (name, caffeine_mg_per_oz) values 
  ('Espresso', 64),
  ('Drip Coffee', 18),
  ('Cold Brew', 24)